<a href="https://colab.research.google.com/github/GeorgeShishkanov/university_project/blob/main/Token_classificayin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!unzip /content/FactRuEval.zip

Archive:  /content/FactRuEval.zip
replace dev.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
!pip install transformers

In [41]:
import pandas as pd
import re
import csv
from datasets import Dataset

In [27]:
DATA = '/content/train.txt'


with open('/content/train.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('train.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)

DATA = pd.read_csv('/content/train.csv', sep='\n')

TOKEN_LIST = []
NER_TAG_LIST = []

tmp1_list = []
tmp2_list = []
list_with_seq = []
list_with_ner = []
count_place = []
count = 0

train_df  = pd.DataFrame()

In [28]:
def element_distributor(element):
    element_after_distributor = element.split(' ')
    token_list.append(element_after_distributor[0])
    ner_tag_list.append(element_after_distributor[1])

In [30]:
data['title,intro'].apply(element_distributor)
print('ready')

ready


In [31]:
for token,tag in zip(token_list, ner_tag_list):
  tmp1_list.append(token)
  tmp2_list.append(tag)
  if token == '.':
    list_with_seq.append(tmp1_list)
    list_with_ner.append(tmp2_list)
    tmp1_list=[]
    tmp2_list=[]
    

In [32]:
train_df['ner_tags'] = list_with_ner
train_df['tokens'] = list_with_seq

In [59]:
train_df

,ner_tags,tokens
0,"[O, O, B-PER, O, O, O, O, O, O, O, O, O, O, O,...","["", Если, Миронов, занял, столь, оппозиционную..."
1,"[O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, B-...","[Источник, "", Ъ, '', в, руководстве, столичной..."
2,"[O, B-LOC, I-LOC, I-LOC, O, O, O, O, B-ORG, B-...","[В, Ханты-Мансийском, автономном, округе, с, д..."
3,"[O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, ...","[С, 1992, года, по, настоящее, время, является..."
4,"[O, O, O, O, O, O, O, O, O, O, B-LOC, I-LOC, O...","[Для, этого, ей, пришлось, выиграть, выборы, в..."
...,...,...
15397,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-P...","[В, свою, очередь, ,, главным, претендентом, н..."
15398,"[O, B-PER, I-PER, O, I-PER, O, O, O, O, O, O, ...","[Канадец, Грег, Снелл, (, Greg, Snell, ), I-PE..."
15399,"[O, O, O, O, O, O, O, O]","[В, префектуре, он, вел, вопросы, социального,..."
15400,"[O, O, O, O, B-PER, I-PER, O, B-ORG, I-ORG, O,...","[Ранее, зампред, экономического, комитета, Вик..."


In [39]:
train_df.isnull().sum()

ner_tags    0
tokens      0
dtype: int64

In [43]:
train_dataset = Dataset.from_pandas(train_df)

In [45]:
train_dataset

Dataset({
    features: ['ner_tags', 'tokens'],
    num_rows: 15402
})

In [47]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
inputs = tokenizer(train_dataset[0]["tokens"], is_split_into_words=True)
inputs.tokens()

In [ ]:
inputs.word_ids()

In [54]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label.format(2) == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [55]:
labels = train_dataset[0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

['O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O']
[-100, 'O', 'O', 'O', 'O', 'O', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [57]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [60]:
tokenized_datasets = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)

  0%|          | 0/16 [00:00<?, ?ba/s]

ArrowInvalid: ignored